In [81]:
# Setup and imports
import os
import warnings
import requests
import pandas as pd
from datetime import datetime, timedelta
import pytz 
import hvplot.pandas  # For advanced visualizations
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi

load_dotenv()


True

In [82]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')


In [83]:
# Define the timezone and get the current date
tz = pytz.timezone('America/New_York')
now = datetime.now(tz)
correct_date = now - timedelta(days=1)
correct_timestamp = pd.Timestamp(correct_date).normalize()
correct_date_str = correct_timestamp.isoformat()

# Set tickers and timeframe
tickers = ["SPY", "QQQ", "IWM", "AAPL", "MSFT", "AMZN", "GOOGL", "META", "TSLA", "NVDA", "JNJ", "JPM"]
timeframe = "1Day"

# Fetch data
current_portfolio_df = api.get_bars(tickers, timeframe, start=correct_date_str, end=correct_date_str).df


In [84]:
# Organize data by dropping 'symbol' and concatenate
if 'symbol' in current_portfolio_df.columns:
    dfs = {ticker: current_portfolio_df[current_portfolio_df['symbol'] == ticker].drop(columns='symbol', errors='ignore') for ticker in tickers}
    portfolio_concat_df = pd.concat(dfs.values(), axis=1, keys=tickers)
    print(portfolio_concat_df.head())
else:
    print("Symbol column not found in DataFrame.")


                              SPY                                      \
                            close    high     low trade_count    open   
timestamp                                                               
2024-05-08 04:00:00+00:00  517.19  517.74  515.14      369154  515.26   

                                                   QQQ                  ...  \
                             volume       vwap   close    high     low  ...   
timestamp                                                               ...   
2024-05-08 04:00:00+00:00  42047115  516.92131  440.06  441.48  437.55  ...   

                              JNJ                          JPM                 \
                             open   volume        vwap   close    high    low   
timestamp                                                                       
2024-05-08 04:00:00+00:00  149.12  7735148  149.037034  195.65  196.65  191.0   

                                                                 

In [85]:
# Define the directory path for raw_data
# 
raw_data_dir = '../data/raw_data'
if not os.path.exists(raw_data_dir):
    os.makedirs(raw_data_dir)

# Function to save DataFrame to CSV
def save_data(df, filename, description):
    if not df.empty:
        df.to_csv(os.path.join(raw_data_dir, filename))
        print(f"{description} data saved successfully to {filename}")
    else:
        print(f"No {description} data to save. Writing null placeholder.")

# Save data
save_data(portfolio_concat_df, 'stock_data.csv', 'Stock') # This code exports portfolio_concat_df to a file named 'stock_data.csv'


Stock data saved successfully to stock_data.csv


In [86]:

# Define the directory path for cleaned_data
cleaned_data_dir = '../data/cleaned_data'
if not os.path.exists(cleaned_data_dir):
    os.makedirs(cleaned_data_dir)

def save_data(df, filename, description):
    if not df.empty:
        df.to_csv(os.path.join(cleaned_data_dir, filename))
        print(f"{description} data saved successfully to {filename}")
    else:
        print(f"No {description} data to save. Writing null placeholder.")
# Save cleaned data
save_data(portfolio_concat_df, 'cleaned_stock_data.csv', 'Clean Stock') # This code exports the same portfolio_concat_df as the one above but not a csv file name 'cleaned_stock_data.csv'


Clean Stock data saved successfully to cleaned_stock_data.csv


In [87]:
# Advanced Data Analysis
close_price = portfolio_concat_df.loc[:, (slice(None), 'close')]
volume = portfolio_concat_df.loc[:, (slice(None), 'volume')]
vwap = portfolio_concat_df.loc[:, (slice(None), 'vwap')]

# Print summary statistics for each stock
print("Summary Statistics:")
for stock in close_price.columns.get_level_values(0).unique():
    print(f"{stock}:")
    print(f"  Closing Price: {close_price.loc[:, (stock, 'close')].values[0]:.2f}")
    print(f"  Volume: {volume.loc[:, (stock, 'volume')].values[0]:,.0f}")
    print(f"  VWAP: {vwap.loc[:, (stock, 'vwap')].values[0]:.2f}")


Summary Statistics:
SPY:
  Closing Price: 517.19
  Volume: 42,047,115
  VWAP: 516.92
QQQ:
  Closing Price: 440.06
  Volume: 24,896,723
  VWAP: 439.77
IWM:
  Closing Price: 203.78
  Volume: 18,349,403
  VWAP: 203.47
AAPL:
  Closing Price: 182.74
  Volume: 45,057,087
  VWAP: 182.48
MSFT:
  Closing Price: 410.54
  Volume: 11,792,308
  VWAP: 410.41
AMZN:
  Closing Price: 188.00
  Volume: 26,136,350
  VWAP: 187.64
GOOGL:
  Closing Price: 169.38
  Volume: 19,586,146
  VWAP: 169.54
META:
  Closing Price: 472.60
  Volume: 11,683,929
  VWAP: 472.02
TSLA:
  Closing Price: 174.72
  Volume: 79,969,488
  VWAP: 173.81
NVDA:
  Closing Price: 904.12
  Volume: 32,572,102
  VWAP: 903.15
JNJ:
  Closing Price: 148.95
  Volume: 7,735,148
  VWAP: 149.04
JPM:
  Closing Price: 195.65
  Volume: 9,227,561
  VWAP: 194.28


In [88]:
portfolio_concat_df

SPY                                      \
                            close    high     low trade_count    open   
timestamp                                                               
2024-05-08 04:00:00+00:00  517.19  517.74  515.14      369154  515.26   

                                                   QQQ                  ...  \
                             volume       vwap   close    high     low  ...   
timestamp                                                               ...   
2024-05-08 04:00:00+00:00  42047115  516.92131  440.06  441.48  437.55  ...   

                              JNJ                          JPM                 \
                             open   volume        vwap   close    high    low   
timestamp                                                                       
2024-05-08 04:00:00+00:00  149.12  7735148  149.037034  195.65  196.65  191.0   

                                                                   
                          trade_count   open   volume        vwap  
timestamp                                                          
2024-05-08 04:00:00+00:00      134463  191.0  9227561  194.278179  

[1 rows x 84 columns]

In [89]:
# Compare closing price with VWAP for each stock and calculate distance from VWAP as a percentage
above_vwap = {}
below_vwap = {}
for stock in portfolio_concat_df.columns.get_level_values(0).unique():
    close_price = portfolio_concat_df.loc[:, (stock, "close")].values[0]
    vwap_price = portfolio_concat_df.loc[:, (stock, 'vwap')].values[0]
    distance_from_vwap_pct = (close_price - vwap_price) / vwap_price * 100
    if close_price > vwap_price:
        above_vwap[stock] = distance_from_vwap_pct
    else:
        below_vwap[stock] = distance_from_vwap_pct

# Print the stocks trading above their VWAP and the distance as a percentage
print("Stocks trading above VWAP:")
for stock, distance_pct in above_vwap.items():
    print(f"{stock}: Above VWAP by {distance_pct:.2f}%")

# Print the stocks trading below their VWAP and the distance as a percentage
print("\nStocks trading below VWAP:")
for stock, distance_pct in below_vwap.items():
    print(f"{stock}: Below VWAP by {distance_pct:.2f}%")

# Insights
print("\nInsights:")
if len(above_vwap) > len(below_vwap):
    print("- The majority of stocks are trading above their VWAP, indicating a bullish sentiment in the market.")
else:
    print("- The majority of stocks are trading below their VWAP, indicating a bearish sentiment in the market.")

max_above_stock = max(above_vwap, key=above_vwap.get)
max_above_distance_pct = above_vwap[max_above_stock]
print(f"- {max_above_stock} has the highest positive distance from VWAP, trading {max_above_distance_pct:.2f}% above its VWAP.")

max_below_stock = min(below_vwap, key=below_vwap.get)
max_below_distance_pct = below_vwap[max_below_stock]
print(f"- {max_below_stock} has the highest negative distance from VWAP, trading {max_below_distance_pct:.2f}% below its VWAP.")

# Convert the dictionaries to DataFrames for easier CSV export
above_vwap_df = pd.DataFrame(list(above_vwap.items()), columns=['Stock', 'Distance_Above_VWAP_Pct'])
below_vwap_df = pd.DataFrame(list(below_vwap.items()), columns=['Stock', 'Distance_Below_VWAP_Pct'])

# Combine both DataFrames for a comprehensive view
combined_vwap_df = pd.concat([above_vwap_df.assign(Above_VWAP=True), below_vwap_df.assign(Above_VWAP=False)])

# Specify the folder path within your project directory where you want to save the CSV
folder_path = '../data/raw_data/'
file_name = 'vwap_analysis.csv'
full_path = f"{folder_path}/{file_name}"

# Export the DataFrame to CSV
combined_vwap_df.to_csv(full_path, index=False)

print(f"Data successfully written to {full_path}")

Stocks trading above VWAP:
SPY: Above VWAP by 0.05%
QQQ: Above VWAP by 0.06%
IWM: Above VWAP by 0.15%
AAPL: Above VWAP by 0.14%
MSFT: Above VWAP by 0.03%
AMZN: Above VWAP by 0.19%
META: Above VWAP by 0.12%
TSLA: Above VWAP by 0.52%
NVDA: Above VWAP by 0.11%
JPM: Above VWAP by 0.71%

Stocks trading below VWAP:
GOOGL: Below VWAP by -0.10%
JNJ: Below VWAP by -0.06%

Insights:
- The majority of stocks are trading above their VWAP, indicating a bullish sentiment in the market.
- JPM has the highest positive distance from VWAP, trading 0.71% above its VWAP.
- GOOGL has the highest negative distance from VWAP, trading -0.10% below its VWAP.
Data successfully written to ../data/raw_data//vwap_analysis.csv


In [90]:
# Calculate total volume for each stock
total_volume = portfolio_concat_df.loc[:, (slice(None), 'volume')].sum()

# Sort the stocks by total volume in descending order
volume_ranking = total_volume.sort_values(ascending=False)

# Format volume numbers with commas
volume_ranking_formatted = volume_ranking.apply(lambda x: f"{x:,.0f}")

# Print the volume ranking
print("Volume Ranking:")
print(volume_ranking_formatted)

# Identify the top 3 most actively traded stocks
top_traded_stocks = volume_ranking.index.get_level_values(0)[:3].tolist()
print(f"\nTop 3 Most Actively Traded Stocks: {', '.join(top_traded_stocks)}")

# Specify the path to the folder within your project folder
folder_path = '../data/raw_data/'
file_name = 'volume_ranking.csv'
full_path = f"{folder_path}/{file_name}"

# Write the formatted volume ranking to a CSV file
volume_ranking_formatted.to_csv(full_path, header=True)

print(f"Data successfully written to {full_path}")

Volume Ranking:
TSLA   volume    79,969,488
AAPL   volume    45,057,087
SPY    volume    42,047,115
NVDA   volume    32,572,102
AMZN   volume    26,136,350
QQQ    volume    24,896,723
GOOGL  volume    19,586,146
IWM    volume    18,349,403
MSFT   volume    11,792,308
META   volume    11,683,929
JPM    volume     9,227,561
JNJ    volume     7,735,148
dtype: object

Top 3 Most Actively Traded Stocks: TSLA, AAPL, SPY
Data successfully written to ../data/raw_data//volume_ranking.csv


In [91]:
# Calculate the average trade size for each security
for stock in portfolio_concat_df.columns.get_level_values(0).unique():
    portfolio_concat_df.loc[:, (stock, 'avg_trade_size')] = portfolio_concat_df.loc[:, (stock, 'volume')] / portfolio_concat_df.loc[:, (stock, 'trade_count')]

# Identify securities with lower trade count compared to volume
institutional_activity = {}
for stock in portfolio_concat_df.columns.get_level_values(0).unique():
    if portfolio_concat_df.loc[:, (stock, 'avg_trade_size')].values[0] > 100:
        institutional_activity[stock] = portfolio_concat_df.loc[:, (stock, 'avg_trade_size')].values[0]

print("Securities with potential institutional activity:")
for stock, avg_trade_size in institutional_activity.items():
    print(f"{stock}: Average trade size = {avg_trade_size:.2f} shares")

# Identify securities with volume closer to trade count
retail_activity = {}
for stock in portfolio_concat_df.columns.get_level_values(0).unique():
    if portfolio_concat_df.loc[:, (stock, 'avg_trade_size')].values[0] <= 100:
        retail_activity[stock] = portfolio_concat_df.loc[:, (stock, 'avg_trade_size')].values[0]

print("\nSecurities with potential retail activity:")
for stock, avg_trade_size in retail_activity.items():
    print(f"{stock}: Average trade size = {avg_trade_size:.2f} shares")


Securities with potential institutional activity:
SPY: Average trade size = 113.90 shares
IWM: Average trade size = 138.57 shares

Securities with potential retail activity:
QQQ: Average trade size = 97.02 shares
AAPL: Average trade size = 86.88 shares
MSFT: Average trade size = 43.27 shares
AMZN: Average trade size = 85.17 shares
GOOGL: Average trade size = 81.58 shares
META: Average trade size = 52.39 shares
TSLA: Average trade size = 88.44 shares
NVDA: Average trade size = 52.43 shares
JNJ: Average trade size = 82.03 shares
JPM: Average trade size = 68.63 shares


In [92]:
import csv

# Assuming the calculations and dictionary creations are done as per your provided code

# Define the folder path where you want to save the CSV files
folder_path = '../data/raw_data/'

# Export institutional activity to CSV
institutional_csv_file = folder_path + 'institutional_activity.csv'
with open(institutional_csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Stock', 'Average Trade Size'])
    for stock, avg_trade_size in institutional_activity.items():
        writer.writerow([stock, avg_trade_size])

print(f"Institutional activity data saved to {institutional_csv_file}")

# Export retail activity to CSV
retail_csv_file = folder_path + 'retail_activity.csv'
with open(retail_csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Stock', 'Average Trade Size'])
    for stock, avg_trade_size in retail_activity.items():
        writer.writerow([stock, avg_trade_size])

print(f"Retail activity data saved to {retail_csv_file}")

Institutional activity data saved to ../data/raw_data/institutional_activity.csv
Retail activity data saved to ../data/raw_data/retail_activity.csv


In [93]:
# Create a copy of the DataFrame for temporary modifications
temp_df = portfolio_concat_df.copy()

# Ensuring column names are strings (useful if DataFrame has MultiIndex columns)
if isinstance(temp_df.columns, pd.MultiIndex):
    temp_df.columns = ['_'.join(col).strip() for col in temp_df.columns.values]

# Now, calculate the high-low range for each stock
# Note: Since we've changed the column names in the copy, we no longer use tuples to reference them
for ticker in tickers:
    high_col = f'{ticker}_high'
    low_col = f'{ticker}_low'
    range_col = f'{ticker}_high_low_range'
    temp_df[range_col] = temp_df[high_col] - temp_df[low_col]

# After fixing the column names in the copy, you can plot without encountering the error
# Here, we need to update the column references for plotting as well
visualization = temp_df.hvplot.bar(
    y=[f'{ticker}_high_low_range' for ticker in tickers],
    xlabel='Date', ylabel='High-Low Price Range', width=800, height=400, legend='top_right', title='Daily Price Range (Volatility)'
).opts(xrotation=45)

# Display the visualization
visualization


:Bars   [timestamp,Variable]   (value)

In [100]:

# Create a temporary DataFrame for plotting
plot_df = portfolio_concat_df.copy()

# Convert MultiIndex column names to string if necessary
if isinstance(plot_df.columns, pd.MultiIndex):
    plot_df.columns = ['_'.join(col).strip() for col in plot_df.columns.values]

# Now, use the modified DataFrame for plotting
# Update the column references for plotting as they are now single-level strings
plot_df.hvplot.bar(
    y=[f'{ticker}_volume' for ticker in tickers],
    xlabel='Date', ylabel='Volume', width=800, height=400, legend='top_right', title='Trading Volume'
).opts(xrotation=45)


:Bars   [timestamp,Variable]   (value)